In [2]:
import pandas as pd
import numpy as np
import math


df = pd.read_csv("joined_understorey_observations.csv")
df



,Year_monitoring,Plot_number,Plot_treatment,Quadrat_number,Quadrat_fenced,Quadrat_gap,Record_ID,T002_Flora_Species_name,Score,Origin,Scientific Name,Common Name,Family Name,Life Form
0,6,1,Gap,1,False,False,10733,Stellaria spp.,0.5,NaN,Stellaria flaccida,Forest Starwort,Caryophyllaceae,Medium Forb/Herb
1,6,1,Gap,2,False,True,10988,Stellaria spp.,0.5,NaN,Stellaria flaccida,Forest Starwort,Caryophyllaceae,Medium Forb/Herb
2,6,1,Gap,3,False,True,11014,Stellaria spp.,0.5,NaN,Stellaria flaccida,Forest Starwort,Caryophyllaceae,Medium Forb/Herb
3,6,1,Gap,4,False,True,11034,Stellaria spp.,1.0,NaN,Stellaria flaccida,Forest Starwort,Caryophyllaceae,Medium Forb/Herb
4,3,1,Gap,4,False,True,119,Stellaria spp.,0.5,NaN,Stellaria flaccida,Forest Starwort,Caryophyllaceae,Medium Forb/Herb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11670,6,12,Gap,17,False,True,15631,Centella cordifolia,0.5,NaN,Centella cordifolia,Centella,Apiaceae,Medium Forb/Herb
11671,3,12,Gap,21,False,False,5562,Acaena ovina,0.5,NaN,Acaena ovina,Australian Sheep's Burr,Rosaceae,Large Forb/Herb
11672,3,12,Gap,22,True,False,5585,Lobelia spp.,0.5,NaN,Lobelia anceps,Angled Lobelia,Campanulaceae,Medium Forb/Herb
11673,3,12,Gap,26,True,False,5663,Lobelia spp.,0.5,NaN,Lobelia anceps,Angled Lobelia,Campanulaceae,Medium Forb/Herb


# Data preparation

In [4]:
groups = df.groupby(["Plot_treatment", "Plot_number", "Quadrat_number", "Year_monitoring"])

print(groups["Record_ID"].count())

print(groups["Scientific Name"].count())

print(groups["Score"])

# count for the ID for each quadrat
(groups["Record_ID"].count()).reset_index().to_csv('grouped.csv', index = False)

grouped_df = pd.read_csv("grouped.csv")
grouped_df

# count for the scientific name for each quadrat with its frequency
name_freq = []
name_count_list = []
for group in groups["Scientific Name"]:
    name_dic = {}
    for item in list(group[1]):
        if item in name_dic.keys():
            name_dic[item] += 1
        else:
            name_dic[item] = 1
    print(name_dic)
    name_count_list.append(len(name_dic.keys()))
    name_freq.append(name_dic)

# formulate the richness
grouped_df["Scientific_Name_count"] = name_count_list

# formulate the diverisity
diveristy = []
for item in name_freq:
    total = 0
    sumup = sum(item.values())
    for value in item.values():
        p = value/sumup
        total += p*math.log(p)
    diveristy.append(total)
grouped_df["Diversity"] = diveristy


# formulate the relative_abundance
grouped_df["Relative_abundance/Eveness"] = grouped_df["Diversity"]/ max(grouped_df["Diversity"])
grouped_df




Plot_treatment  Plot_number  Quadrat_number  Year_monitoring
Control         2            1               0                  18
                                             3                  41
                                             6                  13
                             2               0                  20
                                             3                  62
                                                                ..
Radial          9            31              3                  10
                                             6                   9
                             32              0                  12
                                             3                  21
                                             6                   8
Name: Record_ID, Length: 845, dtype: int64
Plot_treatment  Plot_number  Quadrat_number  Year_monitoring
Control         2            1               0                  18
                               

,Plot_treatment,Plot_number,Quadrat_number,Year_monitoring,Record_ID,Scientific_Name_count,Diversity,Relative_abundance/Eveness
0,Control,2,1,0,18,18,-2.890372,2.630930
1,Control,2,1,3,41,17,-2.774461,2.525423
2,Control,2,1,6,13,13,-2.564949,2.334718
3,Control,2,2,0,20,20,-2.995732,2.726833
4,Control,2,2,3,62,27,-3.209517,2.921428
...,...,...,...,...,...,...,...,...
840,Radial,9,31,3,10,10,-2.302585,2.095903
841,Radial,9,31,6,9,9,-2.197225,2.000000
842,Radial,9,32,0,12,12,-2.484907,2.261860
843,Radial,9,32,3,21,21,-3.044522,2.771244


In [ ]:
# new diversity 

# find shadow tolerant plant and calulate the relative abundance

#  Richness calcualtion

In [5]:
# calculate the absolute richess and relative richness
def find_abs_richness_dataframe(input_df, target):
    diff_list = []
    plot_num = []
    quadrat_num = []
    treatment = []
    i = 0
    while i < len(grouped_df):
        if (grouped_df.iloc[i+1]["Plot_number"] == grouped_df.iloc[i]["Plot_number"]) and (grouped_df.iloc[i+1]["Quadrat_number"] == grouped_df.iloc[i]["Quadrat_number"]) and grouped_df.iloc[i+2]["Year_monitoring"] - grouped_df.iloc[i]["Year_monitoring"] == 6:
            plot_num.append(grouped_df.iloc[i]["Plot_number"])
            quadrat_num.append(grouped_df.iloc[i]["Quadrat_number"])
            treatment.append(grouped_df.iloc[i]["Plot_treatment"])
            diff = grouped_df.iloc[i+2][target] - grouped_df.iloc[i][target]
            diff_list.append(diff)
            i += 3
        else:
            plot_num.append(grouped_df.iloc[i]["Plot_number"])
            quadrat_num.append(grouped_df.iloc[i]["Quadrat_number"])
            treatment.append(grouped_df.iloc[i]["Plot_treatment"])
            diff = grouped_df.iloc[i+1][target]  - grouped_df.iloc[i][target] 
            diff_list.append(diff)
            i += 2
    result = pd.DataFrame({"Treatment":treatment,"Plot_number":plot_num, "Quadrat_number":quadrat_num, "abs_richness":diff_list})
    return result

# calucalte the absolute target value and structure them into an array
def find_abs_target_array(input_df, target):
    temp_list = []
    i = 0
    while i < len(grouped_df):
        if (grouped_df.iloc[i+1]["Plot_number"] == grouped_df.iloc[i]["Plot_number"]) and (grouped_df.iloc[i+1]["Quadrat_number"] == grouped_df.iloc[i]["Quadrat_number"]) and grouped_df.iloc[i+2]["Year_monitoring"] - grouped_df.iloc[i]["Year_monitoring"] == 6:
            diff = grouped_df.iloc[i+2][target] - grouped_df.iloc[i][target] 
            temp_list.append(round(diff,4))
            i += 3
        else:
            diff = grouped_df.iloc[i+1][target] - grouped_df.iloc[i][target] 
            temp_list.append(round(diff,4))
            i += 2
            
    return temp_list

# calucalte the relative target  value and structure them into an array
def find_relative_target_array(input_df, target):
    temp_list = []
    i = 0
    while i < len(grouped_df):
        if (grouped_df.iloc[i+1]["Plot_number"] == grouped_df.iloc[i]["Plot_number"]) and (grouped_df.iloc[i+1]["Quadrat_number"] == grouped_df.iloc[i]["Quadrat_number"]) and grouped_df.iloc[i+2]["Year_monitoring"] - grouped_df.iloc[i]["Year_monitoring"] == 6:
            diff = grouped_df.iloc[i+2][target] /grouped_df.iloc[i][target] 
            temp_list.append(round(diff,4))
            i += 3
        else:
            diff = grouped_df.iloc[i+1][target] /grouped_df.iloc[i][target] 
            temp_list.append(round(diff,4))
            i += 2
            
    return temp_list

result = find_abs_richness_dataframe(grouped_df,"Scientific_Name_count") 
result["relative_richness"] = find_relative_target_array(grouped_df, "Scientific_Name_count")
result["richness_change_rate"] = result["relative_richness"]-1
print(result)


    Treatment  Plot_number  Quadrat_number  abs_richness  relative_richness  \
0     Control            2               1            -5             0.7222   
1     Control            2               2             3             1.1500   
2     Control            2               3             3             1.1765   
3     Control            2               4            -4             0.7647   
4     Control            2               5             0             1.0000   
..        ...          ...             ...           ...                ...   
283    Radial            9              28            -4             0.7333   
284    Radial            9              29            -9             0.5000   
285    Radial            9              30           -12             0.3684   
286    Radial            9              31           -10             0.4737   
287    Radial            9              32            -4             0.6667   

     richness_change_rate  
0                 -0.27

# Diversity calculation

In [6]:
result["abs_diversity"] = find_abs_target_array(grouped_df, "Diversity")
result["relative_diversity"] = find_relative_target_array(grouped_df, "Diversity")
result["diversity_change_rate"] = result["relative_diversity"]-1
print(result)

    Treatment  Plot_number  Quadrat_number  abs_richness  relative_richness  \
0     Control            2               1            -5             0.7222   
1     Control            2               2             3             1.1500   
2     Control            2               3             3             1.1765   
3     Control            2               4            -4             0.7647   
4     Control            2               5             0             1.0000   
..        ...          ...             ...           ...                ...   
283    Radial            9              28            -4             0.7333   
284    Radial            9              29            -9             0.5000   
285    Radial            9              30           -12             0.3684   
286    Radial            9              31           -10             0.4737   
287    Radial            9              32            -4             0.6667   

     richness_change_rate  abs_diversity  relative_

# Relative abundance calculation

In [7]:
result["abs_relative_abundance"] = find_abs_target_array(grouped_df, "Relative_abundance/Eveness")
result["relative_abundance"] = find_relative_target_array(grouped_df, "Relative_abundance/Eveness")
result["relative_abundance_change_rate"] = result["relative_abundance"]-1
print(result)

    Treatment  Plot_number  Quadrat_number  abs_richness  relative_richness  \
0     Control            2               1            -5             0.7222   
1     Control            2               2             3             1.1500   
2     Control            2               3             3             1.1765   
3     Control            2               4            -4             0.7647   
4     Control            2               5             0             1.0000   
..        ...          ...             ...           ...                ...   
283    Radial            9              28            -4             0.7333   
284    Radial            9              29            -9             0.5000   
285    Radial            9              30           -12             0.3684   
286    Radial            9              31           -10             0.4737   
287    Radial            9              32            -4             0.6667   

     richness_change_rate  abs_diversity  relative_

In [8]:
result.to_csv("understorey_output.csv")